# Generating random GORGIAS code

The objective of this notebook is to generate an random but syntaxically correct GORGIAS code, we won't give importance to the semantics behind each code.

In [1]:
import random

actions = [
    # Work-related actions
    "attend_meeting", "finish_report", "reply_emails", "give_presentation",
    
    # Social & leisure activities
    "go_to(restaurant)", "go_to(cinema)", "go_to(park)", "go_to(theater)",
    "visit_family", "attend_concert", "travel_abroad", "go_shopping",

    # Health & exercise
    "go_gym", "morning_run", "yoga_session", "visit_doctor",

    # Daily tasks
    "buy_groceries", "clean_house", "cook_dinner", "read_book",

    # Transportation
    "take_bus", "ride_bike", "drive_car", "book_flight"
]

facts = [
    # Work-related
    "urgent_deadline", "important_meeting", "boss_in_office", "team_project_due",

    # Personal situations
    "feeling_sick", "birthday_today", "wedding_anniversary", "friend_in_town", "medical_appointment",

    # Weather conditions
    "good_weather", "rainy_day", "snowstorm", "hot_day",

    # Time-based events
    "weekend", "holiday_season", "morning_rush", "night_time",

    # Social dynamics
    "invitation_from_friend", "family_gathering", "new_restaurant_to_try", "concert_nearby",

    # Financial considerations
    "low_budget", "got_bonus", "discount_on_flight", "expensive_event"
]


We will classify any GORGIAS as beginner level as the following : 
- simple arguments with clear rules and priorities
- no recursion with minimal dependencies
- a maximum of 1 or 2 layers of preferences
- no `complement()/2`, abducible or defeasible components

An example :
```prolog
rule(r1, go_to_restaurant, []) :- in_holidays.
rule(r2, go_to_cinema, []) :- in_holidays.
rule(p1, prefer(r1, r2), []).
rule(p2, prefer(r2, r1), []) :- new_movie.
rule(c1, prefer(p2, p1), []).
```

In [2]:
def generate_beginner_gorgias():
    action1, action2 = random.sample(actions, 2)
    shared_condition = random.choice(facts)
    
    remaining_facts = [fact for fact in facts if fact != shared_condition]
    p2_condition = random.choice(remaining_facts)

    remaining_facts = [fact for fact in remaining_facts if fact != p2_condition]
    c2_condition = random.choice(remaining_facts)

    rule1 = f"rule(r1, {action1}, []) :- {shared_condition}."
    rule2 = f"rule(r2, {action2}, []) :- {shared_condition}."

    pref1 = f"rule(p1, prefer(r1, r2), [])."

    depth = random.choice([1, 2, 3])  # 1 = just p1, 2 = add p2 & c1, 3 = full depth

    rules = [rule1, rule2, pref1]

    if depth >= 2:
        pref2 = f"rule(p2, prefer(r2, r1), []) :- {p2_condition}."
        conflict1 = "rule(c1, prefer(p2, p1), [])."
        rules.extend([pref2, conflict1])

    if depth == 3:
        conflict2 = f"rule(c2, prefer(p1, p2), []) :- {c2_condition}."
        conflict3 = "rule(c3, prefer(c2, c1), [])."
        rules.extend([conflict2, conflict3])

    return "\n".join(rules)

In [3]:
num_examples = 3
gorgias_examples = [generate_beginner_gorgias() for _ in range(num_examples)]

for i, example in enumerate(gorgias_examples, 1):
    print(f"### Example {i} ###\n{example}\n")

### Example 1 ###
rule(r1, read_book, []) :- night_time.
rule(r2, finish_report, []) :- night_time.
rule(p1, prefer(r1, r2), []).
rule(p2, prefer(r2, r1), []) :- discount_on_flight.
rule(c1, prefer(p2, p1), []).
rule(c2, prefer(p1, p2), []) :- expensive_event.
rule(c3, prefer(c2, c1), []).

### Example 2 ###
rule(r1, travel_abroad, []) :- wedding_anniversary.
rule(r2, attend_meeting, []) :- wedding_anniversary.
rule(p1, prefer(r1, r2), []).
rule(p2, prefer(r2, r1), []) :- concert_nearby.
rule(c1, prefer(p2, p1), []).

### Example 3 ###
rule(r1, read_book, []) :- weekend.
rule(r2, reply_emails, []) :- weekend.
rule(p1, prefer(r1, r2), []).
rule(p2, prefer(r2, r1), []) :- boss_in_office.
rule(c1, prefer(p2, p1), []).



For the intermediate level we will add the `complement()/2`, `neg()/1`, having multi-level preference, more rules (for easy belief theories), for the preference we can add multiple conditions instead of one.

A good intermediate level should be the example of Allow/deny call : 

```prolog
:- dynamic phone_call/0, at_work/0, family_member/1, at_meeting/0.
rule(r1(Call), allow(Call), []):- phone_call.
rule(r2(Call), deny(Call), []):- phone_call.
% Do we need to specify again the phone_call for p1 and p2 ???
rule(p1(Call), prefer(r1(Call), r2(Call)), []):- phone_call.
rule(p2(Call), prefer(r2(Call), r1(Call)), []):- phone_call , at_work.

rule(c1(Call), prefer(p2(Call), p1(Call)), []).
% And here too for at_work ?
rule(c2(Call), prefer(p1(Call), p2(Call)), []):- phone_call , at_work, familly_member(Call).

rule(c3(Call), prefer(c2(Call), c1(Call)), []).

rule(c4(Call), prefer(c1(Call), c2(Call)), []):- phone_call , at_work, familly_member(Call), at_meeting.

rule(c5(Call), prefer(c4(Call), c3(Call)), []).

complement(deny(Call), allow(Call)).
complement(allow(Call), deny(Call)).
```

In [4]:
# WIP
def generate_intermediate_gorgias():
    action1, action2 = random.sample(actions, 2)
    
    fact1 = random.choice(facts)
    remaining_facts = [f for f in facts if f != fact1]
    fact2 = random.choice(remaining_facts)
    remaining_facts = [f for f in remaining_facts if f != fact2]
    fact3 = random.choice(remaining_facts)
    
    rules = []
    
    rule1 = f"rule(r1, {action1}, []) :- {fact1}."
    rule2 = f"rule(r2, {action2}, []) :- {fact1}."
    
    pref1 = f"rule(p1, prefer(r1, r2), []) :- {fact1}, {fact3}."
    pref2 = f"rule(p2, prefer(r2, r1), []) :- {fact2}, {fact3}."
    
    conflict1 = "rule(c1, prefer(p2, p1), [])."
    conflict2 = f"rule(c2, prefer(p1, p2), []) :- {fact2}, neg({fact3})."
    conflict3 = "rule(c3, prefer(c2, c1), [])."
    
    complement1 = f"complement({action2}, {action1})."
    complement2 = f"complement({action1}, {action2})."
    
    rules.extend([rule1, rule2, pref1, pref2, conflict1, conflict2, conflict3, complement1, complement2])
    
    return "\n".join(rules)


In [5]:
intermediate_examples = [generate_intermediate_gorgias() for _ in range(num_examples)]

for i, example in enumerate(intermediate_examples, 1):
    print(f"### Example {i} ###\n{example}\n")

### Example 1 ###
rule(r1, reply_emails, []) :- wedding_anniversary.
rule(r2, go_gym, []) :- wedding_anniversary.
rule(p1, prefer(r1, r2), []) :- wedding_anniversary, snowstorm.
rule(p2, prefer(r2, r1), []) :- medical_appointment, snowstorm.
rule(c1, prefer(p2, p1), []).
rule(c2, prefer(p1, p2), []) :- medical_appointment, neg(snowstorm).
rule(c3, prefer(c2, c1), []).
complement(go_gym, reply_emails).
complement(reply_emails, go_gym).

### Example 2 ###
rule(r1, give_presentation, []) :- urgent_deadline.
rule(r2, attend_concert, []) :- urgent_deadline.
rule(p1, prefer(r1, r2), []) :- urgent_deadline, snowstorm.
rule(p2, prefer(r2, r1), []) :- holiday_season, snowstorm.
rule(c1, prefer(p2, p1), []).
rule(c2, prefer(p1, p2), []) :- holiday_season, neg(snowstorm).
rule(c3, prefer(c2, c1), []).
complement(attend_concert, give_presentation).
complement(give_presentation, attend_concert).

### Example 3 ###
rule(r1, take_bus, []) :- friend_in_town.
rule(r2, read_book, []) :- friend_in_town.
